# bios8366_fp_pw_111418.ipynb

# 11.14.18

In [1]:
%matplotlib inline
import numpy as np
import pymc3 as pm
import pylab as plt
import pandas as pd
import seaborn as sns
sns.set(style="whitegrid")

# Set seed
np.random.seed(10011)

/anaconda2/envs/bios8366/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda2/envs/bios8366/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## LABS

1) create function looking at top X number of labs

In [2]:
# import labs
df_LAB = pd.read_csv('./Data/FONNESBECK_LAB_20151202.csv',sep=',',engine='python',quoting=3)
df_LAB.head()

,RUID,Lab_name,Lab_date,Lab_value
0,50135262,%SAT,04/13/2008,17
1,50135262,ABO,02/08/2007,A
2,50135262,ABO,02/24/2008,a
3,50135262,AN-GAP,02/08/2007,9
4,50135262,AN-GAP,02/11/2011,8


In [3]:
df_LAB1 = df_LAB.copy()

In [4]:
# create function looking at top X number of labs
def top_labs(labs_df,num_labs=10):
    
    """function takes in lab in dataframe format and outputs 
    
        :param labs_df = lab dataframe with columns = {'RUID','Lab_name','Lab_date','Lab_value'}
        :param num_labs = top X of desired top labs to output from function, default = top 10
        
        :return = 
            top_num_labs: pandas dataframe with columns {'Lab_name','counts'} that contains only labs from top number of desired labs
            top_pt_labs: pandas dataframe with columns {'RUID', 'Lab_name', 'Lab_date','Lab_value'} that contains only labs from top number of desired labs
    """
    # preprocess lab dataframe to get sorted labs
    top_labs = pd.DataFrame(labs_df[['Lab_name']].groupby(['Lab_name']).size()).reset_index()
    top_labs.columns = ['Lab_name','counts']
    top_labs1 = top_labs.sort_values(by='counts',ascending=False)
    
    # get dataframe with columns = {'Lab_name', 'counts'}
    top_num_labs = top_labs1.iloc[0:int(num_labs),:]
    
    # create dataframe with only labs matching top labs
    top_pt_labs = labs_df[labs_df['Lab_name'].isin(top_num_labs['Lab_name'])]
    
    return top_num_labs, top_pt_labs

In [5]:
top_num_labs, top_pt_labs = top_labs(df_LAB1,10)

In [6]:
top_num_labs

,Lab_name,counts
1762,GluBed,252113
2218,K,238492
1154,Creat,233493
2609,Na,230471
2776,PCV,229964
592,BUN,226556
1765,Gluc,226554
926,CO2,226256
1070,Cl,226059
2920,Plt-Ct,215131


In [7]:
top_pt_labs.head()

,RUID,Lab_name,Lab_date,Lab_value
121,50135262,BUN,02/08/2007,17
122,50135262,BUN,02/11/2011,25
123,50135262,BUN,02/12/2011,23
124,50135262,BUN,02/13/2007,15
125,50135262,BUN,02/13/2011,17


In [8]:
# write top_pt_labs to csv to read in later
top_pt_labs.to_csv('top_pt_labs_111418.csv',index=False)

In [11]:
# test to see if output was successful
top_pt_labs1 = pd.read_csv('top_pt_labs_111418.csv')
top_pt_labs1.head()

,RUID,Lab_name,Lab_date,Lab_value
0,50135262,BUN,02/08/2007,17
1,50135262,BUN,02/11/2011,25
2,50135262,BUN,02/12/2011,23
3,50135262,BUN,02/13/2007,15
4,50135262,BUN,02/13/2011,17


In [12]:
top_pt_labs['Lab_name'].unique()

array(['BUN', 'CO2', 'Cl', 'Creat', 'GluBed', 'Gluc', 'K', 'Na', 'PCV',
       'Plt-Ct'], dtype=object)

2) explore histograms and remove non-physiological values for labs

In [13]:
glubed_labs = top_pt_labs[top_pt_labs['Lab_name']=='GluBed']

In [14]:
glubed_labs.shape

(252113, 4)

In [15]:
glubed_labs.describe()

,RUID
count,2.521130e+05
mean,5.362519e+07
std,6.056270e+05
min,5.013526e+07
25%,5.372966e+07
50%,5.373197e+07
75%,5.373448e+07
max,5.373642e+07


In [42]:
glubed_labs2 = glubed_labs.copy()

In [46]:
glubed_labs3 = glubed_labs2.sort_values(by='Lab_value',ascending=False)
glubed_labs3.head()

,RUID,Lab_name,Lab_date,Lab_value
3045178,53730650,GluBed,10/23/2013,invalid
767913,53728380,GluBed,09/19/2010,Results
6967417,53734447,GluBed,12/15/2011,Results
1783639,53729342,GluBed,01/08/2010,Results
4613124,53732154,GluBed,12/12/2008,Results


there are bedside glucose values that aren't informative, e.g. "invalid", "results", "ordered", "Credited", ">600", etc.

In [47]:
# identify non-numeric values
glubed_labs3['Lab_value'] = pd.to_numeric(glubed_labs3['Lab_value'],errors='coerce')
glubed_labs3.head()

,RUID,Lab_name,Lab_date,Lab_value
3045178,53730650,GluBed,10/23/2013,NaN
767913,53728380,GluBed,09/19/2010,NaN
6967417,53734447,GluBed,12/15/2011,NaN
1783639,53729342,GluBed,01/08/2010,NaN
4613124,53732154,GluBed,12/12/2008,NaN


In [54]:
glubed_labs4 = glubed_labs3[glubed_labs3['Lab_value'].isna()]

In [55]:
glubed_labs4.shape

(1244, 4)

there are 1244 rows with NaN for `Lab_value`

In [56]:
# count number of unique patients with NaN
glubed_labs4['RUID'].nunique()

489

there 489 unique patients with NaN for `Lab_value`

In [57]:
# remove rows with NaN for Lab_value
glubed_labs5 = glubed_labs3[~glubed_labs3['Lab_value'].isna()]

In [61]:
glubed_labs6 = glubed_labs5.sort_values(by='Lab_value',ascending=False)
glubed_labs6.head()

,RUID,Lab_name,Lab_date,Lab_value
759604,53728363,GluBed,12/24/2013,600.0
1122181,53728649,GluBed,06/18/2014,593.0
2396782,53729950,GluBed,10/02/2013,593.0
8867857,53736205,GluBed,02/17/2014,593.0
8586755,53735988,GluBed,10/13/2014,593.0


In [62]:
# get summary stats for cleaned dataframe
glubed_labs6.describe()

,RUID,Lab_value
count,2.508690e+05,250869.00000
mean,5.362500e+07,159.84525
std,6.061220e+05,74.08201
min,5.013526e+07,12.00000
25%,5.372966e+07,111.00000
50%,5.373197e+07,141.00000
75%,5.373448e+07,189.00000
max,5.373642e+07,600.00000


this cleaned up dataset looks better, in terms of summary stats

In [63]:
# export cleaned-up dataframe to csv
glubed_labs6.to_csv('glubed_labs6.csv',index=False)

In [65]:
# read in csv file as check
glubed_labs_import = pd.read_csv('glubed_labs6.csv')
glubed_labs_import.head()

,RUID,Lab_name,Lab_date,Lab_value
0,53728363,GluBed,12/24/2013,600.0
1,53728649,GluBed,06/18/2014,593.0
2,53729950,GluBed,10/02/2013,593.0
3,53736205,GluBed,02/17/2014,593.0
4,53735988,GluBed,10/13/2014,593.0


## BLOOD-PRESSURE

1) remove non-physiological measures of BP

In [66]:
# import BP data
df_BP = pd.read_csv('./Data/FONNESBECK_BP_20151202.csv',sep=',')
df_BP.head()

,RUID,SYSTOLIC,DIASTOLIC,Measure_date
0,50135262,150,80,01/09/2005
1,50135262,137,77,02/08/2007
2,50135262,137,78,02/08/2007
3,50135262,190,78,02/08/2007
4,50135262,190,77,02/08/2007


In [68]:
df_BP.shape

(10430396, 4)

In [74]:
df_BP['RUID'].nunique()

7484

In [71]:
# identify very high values, most likely inaccurate systolic blood pressures
df_BP1 = df_BP[df_BP['SYSTOLIC']>400]

In [73]:
# see how many unique patients in dataframe
df_BP1['RUID'].nunique()

225

there are 225/7484 patients in dataframe with very high systolic BP

In [83]:
pd.set_option('display.max_rows', 1000)
df_BP1.sort_values(by='SYSTOLIC',ascending=True).tail(1000)

,RUID,SYSTOLIC,DIASTOLIC,Measure_date
6771804,53733372,944,69,08/11/2014
7360064,53733995,944,75,07/01/2012
7360072,53733995,944,64,07/01/2012
7360073,53733995,944,44,07/01/2012
9425034,53735673,944,39,04/13/2007
7360077,53733995,944,60,07/01/2012
7360056,53733995,944,80,07/01/2012
9853531,53736042,944,57,04/18/2013
9972680,53736120,945,70,09/10/2005
3348248,53730650,945,47,06/01/2010


In [75]:
df_BP1.describe()

,RUID,SYSTOLIC,DIASTOLIC
count,1.987000e+03,1987.000000,1987.000000
mean,5.373214e+07,908.807247,54.746855
std,2.537387e+03,111.616889,19.893494
min,5.372791e+07,411.000000,0.000000
25%,5.372984e+07,904.000000,47.000000
50%,5.373229e+07,944.000000,57.000000
75%,5.373442e+07,975.000000,66.500000
max,5.373631e+07,997.000000,155.000000


looking at the summary stats, it may make sense to drop these patients if we use blood pressure

2) another method is to make blood pressures over a certain range `NaN` and perhaps impute them

In [92]:
df_BP_temp = df_BP.copy()

In [95]:
df_BP_temp[df_BP_temp['RUID'].isna()]

,RUID,SYSTOLIC,DIASTOLIC,Measure_date


In [99]:
pd.set_option('display.max_rows', 25)
df_BP_impute = df_BP.copy()
df_BP_impute

,RUID,SYSTOLIC,DIASTOLIC,Measure_date
0,50135262,150,80,01/09/2005
1,50135262,137,77,02/08/2007
2,50135262,137,78,02/08/2007
3,50135262,190,78,02/08/2007
4,50135262,190,77,02/08/2007
5,50135262,137,64,02/08/2007
6,50135262,186,78,02/08/2007
7,50135262,186,64,02/08/2007
8,50135262,190,64,02/08/2007
9,50135262,186,77,02/08/2007


In [100]:
# convert systolic BPs >400 NaN
df_BP_impute['SYSTOLIC'][df_BP_impute['SYSTOLIC']>400] = np.nan
df_BP_impute.head()

/anaconda2/envs/bios8366/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,RUID,SYSTOLIC,DIASTOLIC,Measure_date
0,50135262,150.0,80,01/09/2005
1,50135262,137.0,77,02/08/2007
2,50135262,137.0,78,02/08/2007
3,50135262,190.0,78,02/08/2007
4,50135262,190.0,77,02/08/2007


In [105]:
pd.set_option('display.max_rows', 100)
df_BP_impute.sort_values(by='SYSTOLIC',ascending=True).tail(100)

,RUID,SYSTOLIC,DIASTOLIC,Measure_date
9634703,53735891,NaN,56,11/26/2005
9705983,53735941,NaN,44,06/24/2013
9706025,53735941,NaN,68,06/24/2013
9706034,53735941,NaN,47,06/24/2013
9706057,53735941,NaN,57,06/24/2013
9706066,53735941,NaN,65,06/24/2013
9706077,53735941,NaN,51,06/24/2013
9706084,53735941,NaN,66,06/24/2013
9706126,53735941,NaN,56,06/24/2013
9706138,53735941,NaN,43,06/24/2013


## BMI

1) carry forward impute BMI for NaN values

In [107]:
df_BMI = pd.read_csv('./Data/FONNESBECK_BMI_20151202.csv',sep=',')
df_BMI.head()

,RUID,BMI,Date_BMI,BMI_Weight,BMI_Height,Pregnancy_Indicator
0,50135262,41.43,01/09/2005,119.98,170.18,0
1,50135262,22.86,02/11/2011,70.00,175.00,0
2,50135262,43.07,02/12/2011,131.90,175.00,0
3,50135262,43.06,02/12/2011,131.86,175.00,0
4,50135262,41.47,02/12/2011,127.00,175.00,0


In [109]:
pd.set_option('display.max_rows', 25)
df_BMI[df_BMI['BMI'].isna()]

,RUID,BMI,Date_BMI,BMI_Weight,BMI_Height,Pregnancy_Indicator
744,50135624,NaN,01/21/2013,NaN,177.80,0
1452,50136045,NaN,07/04/2014,NaN,165.10,0
1676,50136072,NaN,12/24/2013,NaN,162.56,0
3108,50138770,NaN,03/14/2013,NaN,167.64,0
4164,50142794,NaN,12/19/2014,NaN,177.80,0
5846,53719335,NaN,09/15/2012,NaN,182.88,0
6902,53726156,NaN,02/21/2013,NaN,154.00,0
7665,53727849,NaN,07/28/2014,NaN,182.00,0
8744,53727891,NaN,08/28/2012,NaN,182.00,0
10512,53727974,NaN,11/10/2013,NaN,118.00,0


In [113]:
pd.set_option('display.max_rows', 50)
df_BMI[df_BMI['RUID']==50135361].head(50)

,RUID,BMI,Date_BMI,BMI_Weight,BMI_Height,Pregnancy_Indicator
71,50135361,27.89,01/07/2011,90.72,180.34,0
72,50135361,27.71,01/09/2012,97.90,187.96,0
73,50135361,27.69,01/10/2012,97.84,187.96,0
74,50135361,27.68,01/10/2012,97.80,187.96,0
75,50135361,31.53,01/15/2007,111.40,187.96,0
76,50135361,31.50,01/19/2007,111.27,187.96,0
77,50135361,33.58,01/22/2005,115.44,185.42,0
78,50135361,34.90,01/25/2003,116.71,182.88,0
79,50135361,33.81,01/28/2005,116.23,185.42,0
80,50135361,30.13,01/29/2009,103.60,185.42,0


there are non-physiological values of BMI for patients who have otherwise physiological BMIs, so would have to impute these values as well

2) explore patients with pregnancy_indicator = 1

In [120]:
df_BMI['RUID'].value_counts()

53728553    2718
53728953    1589
53730165    1450
53734564    1391
53729836    1370
53729875    1277
53731544     914
53735080     875
53728324     755
53735673     744
53731053     686
53735253     681
53735412     667
53733216     664
53734017     606
53733189     534
53731781     415
53732315     391
53736180     390
53732346     369
53732544     367
53734444     366
53730229     349
53736315     347
53730766     340
            ... 
53731373       1
53730341       1
53732898       1
53731619       1
53734946       1
53732342       1
53729050       1
53730763       1
53728289       1
53735569       1
53729056       1
53729906       1
53729636       1
53729569       1
53727845       1
53729268       1
53734948       1
53728101       1
53734442       1
53730998       1
53734067       1
53732911       1
53736110       1
53730906       1
53736124       1
Name: RUID, Length: 6634, dtype: int64

In [117]:
df_BMI_preg = df_BMI[df_BMI['Pregnancy_Indicator']==1]

In [118]:
df_BMI_preg['RUID'].nunique()

608

there are 608/6634 patients with pregnancy_indicator = 1, but most likely these patients are pregnant all of the time, i.e. may be able to use the data 

to do: need to impute NaN and non-physiological values of BMI with carry forward

leave pregnancy_indicator as its own variable, and random forest would be expected to take care of interaction between variables

## EGFR

EGFR: histogram is what we expect during last meeting and is trajectory that they observe when people head towards dialysis to see bump at the end

to-do:  
* may want to use last value in model for various measures
* but this may change to look at trends, i.e. look at variance
* create a helper function that starts dumps out dataframe with 1. last value, 2. median, 3. standard deviation, 4. IQR, 5. date range